### This notebook is to fulfill Project 3 SQL requirement

In [1]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql
import sqlite3
import os

In [2]:
# Postgres info to connect

connection_args = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'dbname': 'names',    # DB that we are connecting to
    'port': 5432          # port we opened on AWS
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)

### Load CSV into pandas dataframe

In [3]:
#read the csv file into panda. the csv file will be stored in the wine database 
#as a dataframe. 
#declare windedf to hold what will become the dataframe.
winedf = pd.read_csv('combined_wine_original.csv')
winedf.head(2)

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,redness
0,0,7.4,0.70,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1


#### Cleaning priorto loading csv into local postgreSQL database

In [4]:
# Drop first column

winedf = winedf.drop(columns='Unnamed: 0')
winedf.head(2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,redness
0,7.4,0.70,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1


In [5]:
# See if any null values
winedf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  redness               6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 660.0 KB


---

### Convert dataframe into table in local postgreSQL called: `wine.db`

In [6]:
# make connection between sql and the wine db
# Name the database
database_name = 'wines.db'

# if this database already exists, then delete it
if database_name in os.listdir():
    os.remove(database_name)

# Now, create and connect to local database 
conn=sqlite3.connect(database_name) 

In [7]:
#This is creating the wines dataframe inside the wines database thru the connection 

# The table inside wines.db is to be called: winetable
winedf.to_sql('winetable', conn)

/opt/miniconda3/envs/metis/lib/python3.7/site-packages/pandas/core/generic.py:2662: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


### EDA via SQL Queries

**Question:** What's the range of quality in these wines?

In [8]:
query = '''
SELECT MAX(quality)-MIN(quality)
FROM winetable
'''

In [9]:
c = conn.cursor()
result = c.execute(query)
for row in result:
    print(row)


(6,)


**Question:** What is the maximum 'acidity' in the highest wine quality?

In [10]:
query = '''
SELECT MAX(`fixed acidity`)
FROM winetable
WHERE quality=9 
'''

In [11]:
c = conn.cursor()
result = c.execute(query)
for row in result:
    print(row)


(9.1,)


**Question:** What is the average 'pH' of red wine? 

In [12]:
query = '''
SELECT AVG(`pH`)
FROM winetable
'''

In [13]:
c = conn.cursor()
result = c.execute(query)
for row in result:
    print(row)

(3.2185008465445644,)


**Question:** How many white wine has 'pH'<4.5 ? 

In [14]:
query = '''
SELECT COUNT(*)
FROM winetable
WHERE redness = 0 AND pH<4.5
'''

In [15]:
c = conn.cursor()
result = c.execute(query)
for row in result:
    print(row)

(4898,)


**Question:** What is the average 'citric acid' content in red & white wine? 

In [16]:
query = '''
SELECT redness, AVG(`citric acid`)
FROM winetable
GROUP BY(redness)

'''

In [17]:
c = conn.cursor()
result = c.execute(query)
for row in result:
    print(row)

(0, 0.33419150673743736)
(1, 0.2709756097560964)


In [18]:
query = '''
SELECT COUNT(*)
FROM winetable
WHERE redness = 0 AND pH<4.5
'''

In [19]:
c = conn.cursor()
result = c.execute(query)
for row in result:
    print(row)

(4898,)


---

### Alternate method to connect to database and load table into pandas dataframe 

In [20]:
# Create your connection.
conn = sqlite3.connect('wines.db')

#declare df to hold the query select all columns from winetable
df = pd.read_sql_query("SELECT * FROM winetable", conn, index_col='index')

In [21]:
#show newly pulled data
#Note the index column is something that sql aoutomatically makes

df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,redness
index,,,,,,,,,,,,,
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


In [22]:
df.shape

(6497, 13)

---

### Define wine Class (TARGET), based on quality

- For this analysis, premium/high-quality wine will be labeled as class (1) & the remainders will be 0
- So, **Premium** wine (Class 1) will be based on quality 7,8,9 and the remainder will be **non-Premium** (Class 0)

In [23]:
#What are the wine ratings?

unique_qualities = sorted(df.quality.unique())
print('unique qualityes:',unique_qualities)

unique qualityes: [3, 4, 5, 6, 7, 8, 9]


In [24]:
df['quality'].value_counts()

6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: quality, dtype: int64

#### Add another column: `class` as TARGET

In [25]:
#Function to code each observation as Premium or Modest class
def class_code(n):
    if n >=7:
        return 1
    else:
        return 0

In [26]:
#Create a column to hold Premium or Modest classes
df['class'] = df['quality'].apply(class_code)

In [27]:
df.head(1)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,redness,class
index,,,,,,,,,,,,,,
0,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1,0


#### Drop column: `Unnamed: 0` as TARGET

In [29]:
#df = df.drop('Unnamed: 0', axis = 'columns')

In [ ]:
df.head(1)

#### Now that the dataframe has a new column that classifies each wine as either Premium or Modest, pickle it.

In [ ]:
import pickle 
with open('combo_wines_cleaned.pickle','wb') as to_write:
    pickle.dump(df, to_write)

NOTE TO SELF:

connection, cursor, fetch, ''' quotes for multiple line queris should all work just like in the 00_pandas_and_postgres file at:

http://localhost:8906/notebooks/OneDrive/Data_Science_Analytics/Metis/sf19_ds15/curriculum/project-03/postgress/00_pandas_and_postgres-Copy1.ipynb#